In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/noaft/transformers.git
!cd transformers/examples/pytorch/question-answering/

Cloning into 'transformers'...
remote: Enumerating objects: 192433, done.
remote: Counting objects: 100% (21419/21419), done.
remote: Compressing objects: 100% (1422/1422), done.
remote: Total 192433 (delta 21030), reused 20004 (delta 19997), pack-reused 171014 (from 1)
Receiving objects: 100% (192433/192433), 210.45 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (139354/139354), done.


In [3]:
!pip install -r /content/transformers/examples/pytorch/question-answering/requirements.txt

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00


In [4]:
%cd transformers
!pip install .

/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9912146 sha256=572d20e1e438d07b8343d306fa248b48c268549be190eacacb35c53b6dc709d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ql7wifti/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [13]:

!python ./examples/pytorch/question-answering/run_seq2seq_qa.py \
  --model_name_or_path /content/drive/MyDrive/Tranning_MT5/checkpoint-12000 \
  --tokenizer_name google/mt5-base \
  --train_file /content/drive/MyDrive/ViQuAD1.0/train_ViQuAD__.json \
  --validation_file /content/drive/MyDrive/ViQuAD1.0/dev_ViQuAD__.json \
  --do_eval \
  --per_device_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 20 \
  --save_steps 1000 \
  --overwrite_output_dir \
  --max_seq_length 584 \
  --doc_stride 128 \
  --output_dir /content/drive/MyDrive/Tranning_MT5/


2024-10-12 14:29:41.361539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 14:29:41.383502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 14:29:41.390204: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 14:29:42.578093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
10/12/2024 14:29:44 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/12/2024 14:29:44 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gp

In [12]:
import torch
from transformers import MT5ForConditionalGeneration, T5Tokenizer

def answer_question(question, context, model, tokenizer):
    # Chuẩn bị input cho mô hình mT5
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate output từ mô hình mT5
    with torch.no_grad():
        output_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)

    # Decode output thành câu trả lời
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Load mô hình mT5 pre-trained và tokenizer
model_name = "/content/drive/MyDrive/Tranning_MT5/checkpoint-12000"  # Hoặc đường dẫn tới mô hình đã huấn luyện
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')

# Ví dụ câu hỏi và ngữ cảnh
question = "Toàn bao nhiêu tuổi?"
context = "Tôi tên Toàn, sinh sống ở Gia Lai, sinh năm 2003 Toàn năm nay học lớp 12 và 18 tuổi"

# Tìm câu trả lời từ mô hình
answer = answer_question(question, context, model, tokenizer)

print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Toàn bao nhiêu tuổi?
Answer: 18 tuổi
